In [1]:
"""
Calculate fire severity statistics within AFD observations from MODIS and VIIRS.
Severity index: Composite Burn Severity (CBI), calculated in GEE via Parks (2018)

Author: maxwell.cook@colorado.edu
"""

import os, sys
import ee
import geemap
import time

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

ee.Authenticate()
ee.Initialize(project='jfsp-aspen')

print("Success !")

Success !


In [6]:
# Load the gridded FRP data
grid = ee.FeatureCollection('projects/jfsp-aspen/assets/viirs_snpp_jpss1_afd_latlon_aspenfires_pixar_gridstats')
print(f"Number of aspen fires: {grid.size().getInfo()}")
grid = grid.select(['grid_index'])
print(grid.first().propertyNames().getInfo())

Number of aspen fires: 49047
['system:index', 'grid_index']


In [7]:
# Load the CBI mosaic
cbi = ee.Image('projects/jfsp-aspen/assets/viirs_snpp_jpss1_aspenfires_srm_cbi_mosaic')
cbi = cbi.select(['CBI_bc','rbr'], ['CBIbc','RBR']) # rename the bands
print(cbi.bandNames().getInfo())

['CBIbc', 'RBR']


In [10]:
# Load the TreeMap data for forest type
treemap = ee.ImageCollection("USFS/GTAC/TreeMap/v2016")
treemap = treemap.select(['FORTYPCD'])
print(f"TreeMap bands available for analysis:\n\n{treemap.first().bandNames().getInfo()}")

TreeMap bands available for analysis:

['FORTYPCD']


In [7]:
# Set up the reductions for CBI statistics in AFD observations and export.

In [11]:
# 1. combined reducer (mean and standard deviation)
average = ee.Reducer.mean().combine(
    reducer2=ee.Reducer.stdDev(), sharedInputs=True)

cbi_average = cbi.reduceRegions(
    collection=grid,
    reducer=average, 
    scale=30
)

# 2. combined reducer (percentiles)
extremes = ee.Reducer.percentile([90,95,99])

cbi_extremes = cbi.reduceRegions(
    collection=grid,
    reducer=extremes, 
    scale=30
)

print("Submitted !")

Submitted !


In [12]:
print(cbi_average.first().propertyNames().getInfo())
print(cbi_extremes.first().propertyNames().getInfo())

['CBIbc_mean', 'CBIbc_stdDev', 'RBR_mean', 'RBR_stdDev', 'system:index', 'grid_index']
['CBIbc_p90', 'CBIbc_p95', 'CBIbc_p99', 'RBR_p90', 'RBR_p95', 'RBR_p99', 'system:index', 'grid_index']


In [13]:
# convoluted ee join method ...
join = ee.Join.inner()
jf = ee.Filter.equals(
    leftField='grid_index', 
    rightField='grid_index'
)
# join the results
cbi_stats = join.apply(cbi_average, cbi_extremes, jf)

# explode the dictionaries ...
def merge_fc(ftr):
    # extract feature properties from join
    primary = ee.Feature(ftr.get('primary')).toDictionary()  # cbi_average
    secondary = ee.Feature(ftr.get('secondary')).toDictionary()  # cbi_extremes
    # combine properties
    merged = primary.combine(secondary)
    return ee.Feature(None, merged)
    
cbi_stats = cbi_stats.map(merge_fc) # expand the merged dictionaries

print(cbi_stats.first().propertyNames().getInfo())

# export table to Drive.
export_task = ee.batch.Export.table.toDrive(
    collection=cbi_stats,
    description='gridstats_cbi_rbr',
    fileNamePrefix='gridstats_cbi_rbr',
    fileFormat='CSV', 
    folder='CBI'
)

export_task.start()
print("Export to Earth Engine Asset started!")
monitor_export(export_task, 120)

['system:index', 'CBIbc_p95', 'RBR_p95', 'grid_index', 'RBR_mean', 'CBIbc_p99', 'RBR_p90', 'RBR_stdDev', 'CBIbc_mean', 'CBIbc_p90', 'CBIbc_stdDev', 'RBR_p99']


In [ ]:
# Calculate CBIbc in forest pixels

In [33]:
# Create a forest mask
import geemap

# Create a forest mask from TreeMap
forest_mask = treemap.select('FORTYPCD').mosaic().gt(0)  # Assuming FORTYPCD > 0 represents forest pixels
masked_cbi = cbi.select(['CBIbc']).updateMask(forest_mask) # mask CBIbc

# Initialize the geemap map
Map = geemap.Map()
Map.add_basemap('SATELLITE')  # Add a satellite basemap for context

# Add the forest mask layer to the map
forest_mask_vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['white', 'green']  # Non-forest pixels in white, forest pixels in green
}
Map.addLayer(forest_mask, forest_mask_vis_params, 'Forest Mask')

# Add the TreeMap layer for comparison (optional)
treemap_vis_params = {
    'min': 0,
    'max': 100,
    'palette': ['white', 'brown', 'darkgreen']  # Visualize the FORTYPCD range
}
Map.addLayer(treemap.select('FORTYPCD'), treemap_vis_params, 'TreeMap')

# Add the masked CBI layer to the map
masked_cbi_vis_params = {
    'min': 0,
    'max': 3,  # Assuming CBI values range from 0 to 3
    'palette': ['blue', 'yellow', 'red']  # Blue for low severity, red for high severity
}
Map.addLayer(masked_cbi, masked_cbi_vis_params, 'Masked CBI')

# Center the map over your area of interest
aoi = grid.geometry().bounds()
Map.centerObject(aoi, zoom=8)
Map.addLayerControl()

# Display the map
Map

Map(center=[38.89676290452681, -107.19536288364709], controls=(WidgetControl(options=['position', 'transparent…

In [32]:
# 1. combined reducer (mean and standard deviation)
average = ee.Reducer.mean().combine(
    reducer2=ee.Reducer.stdDev(), sharedInputs=True)

cbi_average = masked_cbi.reduceRegions(
    collection=grid,
    reducer=average, 
    scale=30
)

# 2. combined reducer (percentiles)
extremes = ee.Reducer.percentile([90,95,99])

cbi_extremes = masked_cbi.reduceRegions(
    collection=grid,
    reducer=extremes, 
    scale=30
)

print("Submitted !")

# convoluted ee join method ...
join = ee.Join.inner()
jf = ee.Filter.equals(
    leftField='grid_index', 
    rightField='grid_index'
)
# join the results
cbi_stats = join.apply(cbi_average, cbi_extremes, jf)

# explode the dictionaries ...
def merge_fc(ftr):
    # extract feature properties from join
    primary = ee.Feature(ftr.get('primary')).toDictionary()  # cbi_average
    secondary = ee.Feature(ftr.get('secondary')).toDictionary()  # cbi_extremes
    # combine properties
    merged = primary.combine(secondary)
    return ee.Feature(None, merged)
    
cbi_stats = cbi_stats.map(merge_fc) # expand the merged dictionaries

print(cbi_stats.first().propertyNames().getInfo())

Submitted !
['system:index', 'CBIbc_p95', 'RBR_p95', 'grid_index', 'RBR_mean', 'CBIbc_p99', 'RBR_p90', 'RBR_stdDev', 'CBIbc_mean', 'CBIbc_p90', 'CBIbc_stdDev', 'RBR_p99']


In [28]:
sample = cbi_stats.limit(10).getInfo()
props = [f['properties'] for f in sample['features']]
df = pd.DataFrame(props)
df.head()

,grid_index,mean,p90,p95,p99,stdDev
0,919906,0.224541,0.90,1.18,1.57,0.392387
1,919907,0.024740,0.10,0.16,0.31,0.080745
2,919908,0.077247,0.22,0.34,0.49,0.113931
3,922166,0.037961,0.16,0.32,0.44,0.101934
4,922171,0.040760,0.17,0.21,0.35,0.085670


In [ ]:
# export table to Drive.
export_task = ee.batch.Export.table.toDrive(
    collection=cbi_stats,
    description='gridstats_cbibc_forest',
    fileNamePrefix='gridstats_cbibc_forest',
    fileFormat='CSV', 
    folder='CBI'
)

export_task.start()
print("Export to Earth Engine Asset started!")
monitor_export(export_task, 120)